<a href="https://colab.research.google.com/github/WEBAAA3/experimentos/blob/main/C%C3%B3pia_drl_pH_recompensa_sigmoid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.23.5
!pip install scipy==1.10.1
!pip install pandas==1.5.3



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3


In [ ]:
!pip install setuptools==65.5.0 "wheel<0.40.0"
!pip install gym==0.21

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.7 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.41.2
    Uninstalling wheel-0.41.2:
      Successfully uninstalled wheel-0.41.2
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
cvxpy 1.3.2 requires setuptools>65.5.1, but you have setuptools 65.5.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616796 sha256=c84a4bebf510f479632e453e5165249aeea2756873ce3c98ddd3bd8f9f66e55a
  Stored in directory: /root/.cache/pip/wheels/81/aa/90/b67df76370d3916a2189b662cf48da38ce41a4e7e58b6abff5
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [ ]:
!pip install stable_baselines3==1.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.8.0
    Uninstalling importlib-metadata-6.8.0:
      Successfully uninstalled importlib-metadata-6.8.0


In [ ]:
import numpy as np
from scipy.integrate import LSODA
class RK5:
    def __init__(self, f, t0, y0, t_final, h):
        #print('correto integrador')
        self.f = f
        self.y = y0
        self.t_final = t_final
        self.h = h
        self.status = 'running'
        self.t = t0

    def step(self):
        if self.t >= self.t_final :
            self.status = 'finished'
            return self.y

        integrator = LSODA(self.f, self.t, self.y, t_bound=self.t+self.h)
        while integrator.status=='running':
            integrator.step()

        self.t+=self.h
        self.y = integrator.y

        return list(self.y)

In [ ]:
import numpy as np
from scipy.integrate import RK45
from scipy.optimize import root
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import os
from collections import OrderedDict
from gym import spaces
from gym.utils import seeding
import gym


from stable_baselines3.common.utils import get_device
get_device(device='cuda')

device(type='cuda')

In [ ]:
x_T_min = [25]*6 + [20]*7 + [30]*6
y_T_min = [10.8 , 18 , 14.4981688 , 22.9943 , 24.61175 , 20 , 8.73682593 , 18 , 14.4976726 , 22.49393 , 24.612692 , 20 , 11 , 12.9033455 , 18 , 14.4987935 , 24.6108 , 16 , 20]
z_T_min = [25.89 , 27.7977 , 26.75647 , 30.34558 , 35.30862 , 28.572924 , 21.7697 , 25.379 , 23.83778 , 28.127 , 35.12 , 26.42 , 22.5252 , 30.0623296 , 30.5471 , 30.16534 , 35.495354 , 30.27772185 , 30.96356]

x_T_max = [25]*6 + [20]*7 + [30]*6
y_T_max = [10.9 , 18 , 14.76477 , 23 , 16 , 20 , 8.736826 , 18 , 14.5087475 , 24.56211845 , 16 , 20 , 11.59126 , 12.9085 , 18 , 14.7185 , 24.5018754 , 16.1222 , 20]
z_T_max = [26.2573 , 39.0517 , 33.79362 , 39.63658 , 36.43654 , 39.6419 , 21.7697 , 35.60426 , 30.9273 , 35.53598 , 32.7886 , 38.9965 , 27.766 , 30.06545 , 39.3761 , 33.6998 , 35.78658 , 36.606544 , 39.6498544]

from scipy.interpolate import griddata
import numpy as np

def interpola_spline_cubico(x, y, z, xi, yi):
    # Converte as listas x, y, e z para arrays numpy
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    xi, yi = np.meshgrid(xi, yi)

    # Realiza a interpolação por spline cúbico
    zi = griddata((x, y), z, (xi, yi), method='cubic')

    return zi

def get_T_min(T_in, cp):
    return interpola_spline_cubico(x_T_min, y_T_min, z_T_min, [T_in], [cp])[0][0]
def get_T_max(T_in, cp):
    return interpola_spline_cubico(x_T_max, y_T_max, z_T_max, [T_in], [cp])[0][0]

print(get_T_min(25,18))
print(get_T_max(25,18))


27.7977
39.0517


In [ ]:
from scipy.integrate import RK45
from scipy.optimize import root
from scipy.optimize import minimize
from scipy import stats
from gym import spaces
from gym.utils import seeding
import gym


class eval_bioreactor(gym.Env):

    metadata = {
        'render.modes' : ['human', 'rgb_array'],
        'video.frames_per_second' : 30
    }

    def __init__(self):

        # Constants
        # Specific Ionic constants     (l/g_ion)
        self.HNa = -0.550
        self.HCa = -0.303
        self.HMg = -0.314
        self.HH = -0.774
        self.HCl = 0.844
        self.HCO3 = 0.485
        self.HHO = 0.941
        # Molecular masses     (g/mol)
        self.MNaCl = 58.5
        self.MCaCO3 = 90
        self.MMgCl2 = 95
        self.MNa = 23
        self.MCa = 40
        self.MMg = 24
        self.MCl = 35.5
        self.MCO3 = 60
        # Kinetic constants
        # miu_X  = 0.556;      % [1/h]
        self.miu_P = 1.790  # [1/h]
        self.Ks = 1.030  # [g/l]
        self.Ks1 = 1.680  # [g/l]
        self.Kp = 0.139  # [g/l]
        self.Kp1 = 0.070  # [g/l]
        self.Rsx = 0.607
        self.Rsp = 0.435
        self.YO2 = 0.970  # [mg/mg]
        self.KO2 = 8.86  # [mg/l]
        self.miu_O2 = 0.5  # [1/h]
        self.A1 = 9.5e8
        self.A2 = 2.55e33
        self.Ea1 = 55000  # J/mol
        self.Ea2 = 220000  # J/mol
        self.R = 8.31  # J/(mol.K)
        # thermodynamic constants
        self.Kla0 = 38  # [1/h]
        self.KT = 100 * 3600  # [J/hm2K]
        self.Vm = 50  # [l]
        self.AT = 1  # [m2]
        self.ro = 1080  # [g/l]
        self.ccal = 4.18  # [J/gK]
        self.roag = 1000  # [g/l]
        self.ccalag = 4.18  # [J/gK]
        self.deltaH = 518  # [kJ/mol O2 consumat]

        # Initial data
        self.mNaCl = 500  # [g]
        self.mCaCO3 = 100  # [g]
        self.mMgCl2 = 100  # [g]
        self.pH = 6
        self.cH = 10**(-self.pH)
        self.cOH = 10**(-(14 - self.pH))
        self.Tiag = 15  # [°C]

        # ex u
        #self.Fi = 0  # l/h
        #self.Fe = self.Fi  # l/h
        #self.T_in = 25  # oC
        self.cS_in = 60  # g/l


        #parametros para o RL
        self.observation_space = spaces.Box(low=np.array([-81.,-2160.,-121]),
                                          high=np.array([ +77.,+1235.,+100]),
                                          dtype=np.float32)

        self.action_space = spaces.Box(low=np.array([-1.,-1.,-1.]),
                                       high=np.array([+1.,+1.,+1.]),
                                       shape=(3,), dtype=np.float32)
        self.best_cum_reward=-1e6
        self.cum_reward=0
        self.current_step = 0
        self.seed()
        self.erro_vec_T=[]
        self.erro_vec_Cetoh=[]
        self.erro_quadratico_T=[]
        self.erro_quadratico_Cetoh=[]
        self.actions_faq = []
        self.actions_Fi = []
        self.actions_Fe = []

        self.episode=0
        self.steps_per_ep = []

        self.Ab = 3.14159*1000
        self.h0 = 0.2228-0.01274
        self.cX0 = 0.904676
        self.cP0 = 18-1.8
        self.cS0 = 29.1749
        self.cO20 = 3.10695
        self.T0 = 30 - 2
        self.Tag0 = 27

        self.K_t = 695.05
        self.K_oh = 336.39
        self.K_h = 17894.2

        self.time_span = 2
        self.t_final = 250
        self.reset()


    def seed(self, seed=1):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]



    def differentials(self, t, y):
        y = [i if i>0 else 0 for i in y]
        h = y[0]
        V = self.Ab*h
        cX = y[1]
        cP = y[2]
        cS = y[3]
        cO2 = y[4]
        T = y[5]
        Tag = y[6]

        Fag = self.faq_RL

        # Calculating concentration of species and ionic strengths
        c0st = 14.16 - 0.3943 * T + 0.007714 * T**2 - 0.0000646 * T**3  # [mg/l]
        cNa = self.mNaCl / self.MNaCl * self.MNa / V
        cCa = self.mCaCO3 / self.MCaCO3 * self.MCa / V
        cMg = self.mMgCl2 / self.MMgCl2 * self.MMg / V
        cCl = (self.mNaCl / self.MNaCl + 2 * self.mMgCl2 / self.MMgCl2) * self.MCl / V
        cCO3 = self.mCaCO3 / self.MCaCO3 * self.MCO3 / V

        INa = 0.5 * cNa * ((+1)**2)
        ICa = 0.5 * cCa * ((+2)**2)
        IMg = 0.5 * cMg * ((+2)**2)
        ICl = 0.5 * cCl * ((-1)**2)
        ICO3 = 0.5 * cCO3 * ((-2)**2)
        IH = 0.5 * self.cH * ((+1)**2)
        IOH = 0.5 * self.cOH * ((-1)**2)
        sumaHiIi = self.HNa * INa + self.HCa * ICa + self.HMg * IMg + self.HCl * ICl + self.HCO3 * ICO3 + self.HH * IH + self.HHO * IOH
        cst = c0st * 10**(-sumaHiIi)
        alfa = 0.8
        Kla = self.Kla0 * (1.024**(T - 20))

        rO2 = 1000*self.miu_O2 * cO2 * cX / (self.YO2 * (self.KO2 + cO2)) # mg/lh
        #print('rO2',self.miu_O2, cO2, cX, self.YO2, self.KO2,cO2)

        # 0,5*3,1*900 / (0,97*(8,86+900)
        #/h * mg/l * g/l / mg/l
        miu_X = self.A1 * np.exp(-self.Ea1 / self.R / (T + 273)) - self.A2 * np.exp(-self.Ea2 / self.R / (T + 273))

        # Differential equations
        dh = (self.Fi - self.Fe)/self.Ab
        dcX = (miu_X * cX * cS / (self.Ks + cS)) * np.exp(-self.Kp * cP) - (self.Fe / V) * cX  # g/(l.h)

        dcP = (self.miu_P * cX * cS / (self.Ks1 + cS)) * np.exp(-self.Kp1 * cP) - (self.Fe / V) * cP  # g/(l.h)
        dcS =((-miu_X*cX*cS* np.exp(-self.Kp * cP))/(self.Rsx*(self.Ks + cS)) +
              (-self.miu_P*cX*cS* np.exp(-self.Kp1 * cP))/(self.Rsp*(self.Ks1 + cS)) +
              ((self.Fi / V) * self.cS_in) +
              (- (self.Fe / V) * cS))


        dcO2 = Kla * (cst - cO2) - rO2 - (self.Fe / V) * cO2  # mg/(l.h)


        dT = ( (self.Fi*(self.T_in+273)/V)+
               (-self.Fe*(T+273)/V)+
               ( rO2*self.deltaH/(32*self.ro*self.ccal))+
               (-self.KT*self.AT*(T-Tag)/(V*self.ro*self.ccal))
        )


        dTag = ((Fag*(self.Tiag-Tag)/self.Vm) +
                (self.KT*self.AT*(T-Tag)/(self.Vm*self.roag*self.ccalag))
               )
        return [dh, dcX, dcP, dcS, dcO2, dT, dTag]


    def save_traj(self, step, rw):
        traj = pd.DataFrame(
            {'erro': self.erro_vec_pH[:-1],
             'acao_qa': self.actions_qa,
             'acao_qb': self.actions_qb,
             'pH': self.pH_vec,
             'rw': self.rw_vec
            })
        log_dir = "{0}EP_{1}_STEPS{2}_REWARD_{3}_ISE/".format(self.episode,
                                                              step,
                                                              rw,
                                                              int(sum(self.erro_quadratico_pH)))
        os.makedirs(log_dir, exist_ok=True)
        path=log_dir+'/traj_sim_RL.csv'
        traj.to_csv(path)

    def reset(self):
        #ADAPTATIVO
        self.T_in = 30
        self.set_point_etoh = 18

        self.set_point_T = 30
        self.set_point_h =0.2228



        if self.cum_reward>self.best_cum_reward and self.cum_reward!=0:
            self.best_cum_reward = self.cum_reward

        self.steps_per_ep.append(self.current_step)
        self.current_step = 0
        if len(self.steps_per_ep)==10:
            #print('média de steps/ep: ', np.mean(self.steps_per_ep))
            self.steps_per_ep=[]


        self.cum_reward=0
        self.erro_quadratico_T=[]
        self.erro_quadratico_Cetoh=[]
        self.t = []
        self.actions_faq = []
        self.actions_Fi = []
        self.actions_Fe = []

        self.yeast=[]
        self.etoh=[]
        self.glic=[]
        self.oxy=[]
        self.T_jacket=[]
        self.rw_vec_T=[]
        self.rw_vec_etoh=[]
        self.rw_vec_h=[]
        self.rw_vec=[]
        self.acao_faq=25 #l/h
        self.acao_Fi=25
        self.acao_Fe=25


        f = self.differentials
        t0 = 0
        y0 = [self.h0, self.cX0, self.cP0, self.cS0, self.cO20, self.T0, self.Tag0]
        t_final = self.t_final
        dt = self.time_span

        self.integrator = RK5(f, t0, y0, t_final, dt)
        self.episode+=1

        self.erro_vec_T = [((y0[-2] - self.set_point_T)*self.K_t)]
        self.erro_vec_etoh = [((y0[2] - self.set_point_etoh)*self.K_oh)]
        self.erro_vec_h = [((y0[0] - self.set_point_h)*self.K_h)]

        #print([self.erro_vec_T[-1], self.erro_vec_h[-1], self.erro_vec_etoh[-1]] )
        return [self.erro_vec_T[-1], self.erro_vec_h[-1], self.erro_vec_etoh[-1]]

    def compute_reward(self,erro_atual):
        rw = abs(erro_atual)/(self.K_t+self.K_oh+self.K_h)

        return rw


    def step(self, action):

        self.t.append(self.integrator.t)
        if self.episode in [5*(i) + 1 for i in range(1000)]:
            if self.t[-1]<100:
                self.set_point_T = 30
            elif self.t[-1]>=100 and self.t[-1]<150:
                self.set_point_T = 29
            elif self.t[-1]>=150 and self.t[-1]<200:
                self.set_point_T = 32
            else:
                self.set_point_T = 34

        elif self.episode in [5*(i) + 2 for i in range(1000)]:
            if self.t[-1]<100:
                self.set_point_etoh = 18
            elif self.t[-1]>=100 and self.t[-1]<150:
                self.set_point_etoh = 15
            elif self.t[-1]>=150 and self.t[-1]<200:
                self.set_point_etoh = 20
            else:
                self.set_point_etoh = 23

        elif self.episode in [5*(i) + 3 for i in range(1000)]:
            if self.t[-1]<100:
                self.set_point_h = 0.2228
            elif self.t[-1]>=100 and self.t[-1]<150:
                self.set_point_h = 0.2164
            elif self.t[-1]>=150 and self.t[-1]<200:
                self.set_point_h = 0.2292
            else:
                self.set_point_h = 0.2355

        elif self.episode in [5*(i) + 4 for i in range(1000)]:
            if self.t[-1]<100:
                self.T_in = 28
            elif self.t[-1]>=100 and self.t[-1]<150:
                self.T_in = 30
            elif self.t[-1]>=150 and self.t[-1]<200:
                self.T_in = 26
            else:
                self.T_in = 24

        self.acao_faq += action[0]*5
        self.acao_faq =min([max([0, self.acao_faq]),50])
        self.actions_faq.append(self.acao_faq)
        self.faq_RL = self.acao_faq

        self.acao_Fi += action[1]*5
        self.acao_Fi =min([max([0, self.acao_Fi]),50])
        self.actions_Fi.append(self.acao_Fi)
        self.Fi = self.acao_Fi

        self.acao_Fe += action[2]*5
        self.acao_Fe =min([max([0, self.acao_Fe]),50])
        self.actions_Fe.append(self.acao_Fe)
        self.Fe = self.acao_Fe

        self.integrator.step()
        self.t.append(self.integrator.t)
        self.current_step += 1
        y = self.integrator.y
        #print('---', y[-2], y[2], y[0])
        self.erro_vec_T.append((y[-2] - self.set_point_T)*self.K_t)
        self.erro_vec_etoh.append((y[2] - self.set_point_etoh)*self.K_oh)
        self.erro_vec_h.append((y[0] - self.set_point_h)*self.K_h)

        reward_T = self.compute_reward(self.erro_vec_T[-1])
        reward_etoh = self.compute_reward(self.erro_vec_etoh[-1])
        reward_h = self.compute_reward(self.erro_vec_h[-1])

        reward =  12-((reward_T+ reward_etoh+ reward_h)/0.11749)

        self.rw_vec_T.append(reward_T)
        self.rw_vec_etoh.append(reward_etoh)
        self.rw_vec_h.append(reward_h)
        self.rw_vec.append(reward)
        self.yeast.append(self.integrator.y[1])

        self.glic.append(self.integrator.y[3])
        self.oxy.append(self.integrator.y[4])
        self.T_jacket.append(self.integrator.y[6])
        self.cum_reward+=reward
        #print(action,self.acao_faq, self.Fi, self.Fe)
        if (self.integrator.status != 'running' or
            y[-2]>50 or y[-2]<10 or
            y[0]>0.2865 or y[0]<0.0637
           ):
            done = True
            """
            if self.episode in[5*(i) + 1 for i in range(1000)]:
                print('333333333333333333333333333333333333333333333- servo T -  ', self.current_step)
            elif self.episode in[5*(i) + 2 for i in range(1000)]:
                print('333333333333333333333333333333333333333333333- servo etoh -  ', self.current_step)
            elif self.episode in[5*(i) + 3 for i in range(1000)]:
                print('333333333333333333333333333333333333333333333- servo h -  ', self.current_step)
            elif self.episode in[5*(i) + 4 for i in range(1000)]:
                print('333333333333333333333333333333333333333333333- regulatorio -  ', self.current_step)
            else:
                print('333333333333333333333333333333333333333333333- constante -  ', self.current_step)


            print('IAE_k T: ',sum([abs(i) for i in self.erro_vec_T]))
            print('IAE_k etoh: ',sum([abs(i) for i in self.erro_vec_etoh]))
            print('IAE_k h: ',sum([abs(i) for i in self.erro_vec_h]))
            print('done',self.current_step, y[-2], y[2], y[0])
            """
        else:
            done = False

        obs =  [self.erro_vec_T[-1], self.erro_vec_h[-1], self.erro_vec_etoh[-1]]


        info = {'is_success': done}
        return obs, reward, done, info


    def render(self, mode='human'):
        print("Not implemented")
        return

    def close(self):
        print("Not implemented")
        return





In [ ]:
import warnings
warnings.filterwarnings("ignore")
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy


def reset_eval():
    eval_env = eval_bioreactor()
    return eval_env



In [ ]:
from typing import Callable
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [ ]:
class bioreactor(gym.Env):

    metadata = {
        'render.modes' : ['human', 'rgb_array'],
        'video.frames_per_second' : 30
    }

    def __init__(self):

        # Constants
        # Specific Ionic constants     (l/g_ion)
        self.HNa = -0.550
        self.HCa = -0.303
        self.HMg = -0.314
        self.HH = -0.774
        self.HCl = 0.844
        self.HCO3 = 0.485
        self.HHO = 0.941
        # Molecular masses     (g/mol)
        self.MNaCl = 58.5
        self.MCaCO3 = 90
        self.MMgCl2 = 95
        self.MNa = 23
        self.MCa = 40
        self.MMg = 24
        self.MCl = 35.5
        self.MCO3 = 60
        # Kinetic constants
        # miu_X  = 0.556;      % [1/h]
        self.miu_P = 1.790  # [1/h]
        self.Ks = 1.030  # [g/l]
        self.Ks1 = 1.680  # [g/l]
        self.Kp = 0.139  # [g/l]
        self.Kp1 = 0.070  # [g/l]
        self.Rsx = 0.607
        self.Rsp = 0.435
        self.YO2 = 0.970  # [mg/mg]
        self.KO2 = 8.86  # [mg/l]
        self.miu_O2 = 0.5  # [1/h]
        self.A1 = 9.5e8
        self.A2 = 2.55e33
        self.Ea1 = 55000  # J/mol
        self.Ea2 = 220000  # J/mol
        self.R = 8.31  # J/(mol.K)
        # thermodynamic constants
        self.Kla0 = 38  # [1/h]
        self.KT = 100 * 3600  # [J/hm2K]
        self.Vm = 50  # [l]
        self.AT = 1  # [m2]
        self.ro = 1080  # [g/l]
        self.ccal = 4.18  # [J/gK]
        self.roag = 1000  # [g/l]
        self.ccalag = 4.18  # [J/gK]
        self.deltaH = 518  # [kJ/mol O2 consumat]

        # Initial data
        self.mNaCl = 500  # [g]
        self.mCaCO3 = 100  # [g]
        self.mMgCl2 = 100  # [g]
        self.pH = 6
        self.cH = 10**(-self.pH)
        self.cOH = 10**(-(14 - self.pH))
        self.Tiag = 15  # [°C]

        # ex u
        #self.Fi = 0  # l/h
        #self.Fe = self.Fi  # l/h
        #self.T_in = 25  # oC
        self.cS_in = 60  # g/l


        #parametros para o RL
        self.observation_space = spaces.Box(low=np.array([-81.,-2160.,-121]),
                                          high=np.array([ +77.,+1235.,+100]),
                                          dtype=np.float32)

        self.action_space = spaces.Box(low=np.array([-1.,-1.,-1.]),
                                       high=np.array([+1.,+1.,+1.]),
                                       shape=(3,), dtype=np.float32)
        self.best_cum_reward=-1e6
        self.cum_reward=0
        self.current_step = 0
        self.seed()
        self.erro_vec_T=[]
        self.erro_vec_Cetoh=[]
        self.erro_quadratico_T=[]
        self.erro_quadratico_Cetoh=[]
        self.actions_faq = []
        self.actions_Fi = []
        self.actions_Fe = []

        self.episode=0
        self.steps_per_ep = []

        self.Ab = 3.14159*1000
        self.h0 = 0.2228-0.01274
        self.cX0 = 0.904676
        self.cP0 = 18-1.8
        self.cS0 = 29.1749
        self.cO20 = 3.10695
        self.T0 = 30 - 2
        self.Tag0 = 27

        self.K_t = 695.05
        self.K_oh = 336.39
        self.K_h = 17894.2

        self.time_span = 2
        self.t_final = 250
        self.reset()


    def seed(self, seed=1):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]



    def differentials(self, t, y):
        y = [i if i>0 else 0 for i in y]
        h = y[0]
        V = self.Ab*h
        cX = y[1]
        cP = y[2]
        cS = y[3]
        cO2 = y[4]
        T = y[5]
        Tag = y[6]

        Fag = self.faq_RL

        # Calculating concentration of species and ionic strengths
        c0st = 14.16 - 0.3943 * T + 0.007714 * T**2 - 0.0000646 * T**3  # [mg/l]
        cNa = self.mNaCl / self.MNaCl * self.MNa / V
        cCa = self.mCaCO3 / self.MCaCO3 * self.MCa / V
        cMg = self.mMgCl2 / self.MMgCl2 * self.MMg / V
        cCl = (self.mNaCl / self.MNaCl + 2 * self.mMgCl2 / self.MMgCl2) * self.MCl / V
        cCO3 = self.mCaCO3 / self.MCaCO3 * self.MCO3 / V

        INa = 0.5 * cNa * ((+1)**2)
        ICa = 0.5 * cCa * ((+2)**2)
        IMg = 0.5 * cMg * ((+2)**2)
        ICl = 0.5 * cCl * ((-1)**2)
        ICO3 = 0.5 * cCO3 * ((-2)**2)
        IH = 0.5 * self.cH * ((+1)**2)
        IOH = 0.5 * self.cOH * ((-1)**2)
        sumaHiIi = self.HNa * INa + self.HCa * ICa + self.HMg * IMg + self.HCl * ICl + self.HCO3 * ICO3 + self.HH * IH + self.HHO * IOH
        cst = c0st * 10**(-sumaHiIi)
        alfa = 0.8
        Kla = self.Kla0 * (1.024**(T - 20))

        rO2 = 1000*self.miu_O2 * cO2 * cX / (self.YO2 * (self.KO2 + cO2)) # mg/lh
        #print('rO2',self.miu_O2, cO2, cX, self.YO2, self.KO2,cO2)

        # 0,5*3,1*900 / (0,97*(8,86+900)
        #/h * mg/l * g/l / mg/l
        miu_X = self.A1 * np.exp(-self.Ea1 / self.R / (T + 273)) - self.A2 * np.exp(-self.Ea2 / self.R / (T + 273))

        # Differential equations
        dh = (self.Fi - self.Fe)/self.Ab
        dcX = (miu_X * cX * cS / (self.Ks + cS)) * np.exp(-self.Kp * cP) - (self.Fe / V) * cX  # g/(l.h)

        dcP = (self.miu_P * cX * cS / (self.Ks1 + cS)) * np.exp(-self.Kp1 * cP) - (self.Fe / V) * cP  # g/(l.h)
        dcS =((-miu_X*cX*cS* np.exp(-self.Kp * cP))/(self.Rsx*(self.Ks + cS)) +
              (-self.miu_P*cX*cS* np.exp(-self.Kp1 * cP))/(self.Rsp*(self.Ks1 + cS)) +
              ((self.Fi / V) * self.cS_in) +
              (- (self.Fe / V) * cS))


        dcO2 = Kla * (cst - cO2) - rO2 - (self.Fe / V) * cO2  # mg/(l.h)


        dT = ( (self.Fi*(self.T_in+273)/V)+
               (-self.Fe*(T+273)/V)+
               ( rO2*self.deltaH/(32*self.ro*self.ccal))+
               (-self.KT*self.AT*(T-Tag)/(V*self.ro*self.ccal))
        )


        dTag = ((Fag*(self.Tiag-Tag)/self.Vm) +
                (self.KT*self.AT*(T-Tag)/(self.Vm*self.roag*self.ccalag))
               )
        return [dh, dcX, dcP, dcS, dcO2, dT, dTag]


    def save_traj(self, step, rw):
        traj = pd.DataFrame(
            {'erro': self.erro_vec_pH[:-1],
             'acao_qa': self.actions_qa,
             'acao_qb': self.actions_qb,
             'pH': self.pH_vec,
             'rw': self.rw_vec
            })
        log_dir = "{0}EP_{1}_STEPS{2}_REWARD_{3}_ISE/".format(self.episode,
                                                              step,
                                                              rw,
                                                              int(sum(self.erro_quadratico_pH)))
        os.makedirs(log_dir, exist_ok=True)
        path=log_dir+'/traj_sim_RL.csv'
        traj.to_csv(path)

    def reset(self):
        #ADAPTATIVO
        self.T_in = np.random.uniform(low=20, high=30, size=(1,))[0]
        self.set_point_etoh = np.random.uniform(low= (0.425*self.T_in+0.2917)*1.01,
                                                high=24,
                                                size=(1,))[0]

        self.set_point_T = np.random.uniform(low=get_T_min(self.T_in,self.set_point_etoh)*1.01,
                                             high=get_T_max(self.T_in,self.set_point_etoh)*0.99,
                                             size=(1,))[0]
        self.set_point_h = np.random.uniform(low=0.1592, high=0.2865, size=(1,))[0]

        #print(self.T_in,
        #      self.set_point_T,
        #      self.set_point_etoh,
        #      self.set_point_h)

        if self.cum_reward>self.best_cum_reward and self.cum_reward!=0:
            self.best_cum_reward = self.cum_reward

        self.steps_per_ep.append(self.current_step)
        self.current_step = 0
        if len(self.steps_per_ep)==100:
            #print('média de steps/ep: ', np.mean(self.steps_per_ep))
            self.steps_per_ep=[]


        self.cum_reward=0
        self.erro_quadratico_T=[]
        self.erro_quadratico_Cetoh=[]
        self.t = []
        self.actions_faq = []
        self.actions_Fi = []
        self.actions_Fe = []

        self.yeast=[]
        self.etoh=[]
        self.glic=[]
        self.oxy=[]
        self.T_jacket=[]
        self.rw_vec_T=[]
        self.rw_vec_etoh=[]
        self.rw_vec_h=[]
        self.rw_vec=[]
        self.acao_faq=25 #l/h
        self.acao_Fi=25
        self.acao_Fe=25


        f = self.differentials
        t0 = 0
        y0 = [self.h0, self.cX0, self.cP0, self.cS0, self.cO20, self.T0, self.Tag0]
        t_final = self.t_final
        dt = self.time_span

        self.integrator = RK5(f, t0, y0, t_final, dt)
        self.episode+=1

        self.erro_vec_T = [((y0[-2] - self.set_point_T)*self.K_t)]
        self.erro_vec_etoh = [((y0[2] - self.set_point_etoh)*self.K_oh)]
        self.erro_vec_h = [((y0[0] - self.set_point_h)*self.K_h)]

        return [self.erro_vec_T[-1], self.erro_vec_h[-1], self.erro_vec_etoh[-1]]

    def compute_reward(self,erro_atual):
        rw = abs(erro_atual)/(self.K_t+self.K_oh+self.K_h)

        return rw

    def step(self, action):

        self.acao_faq += action[0]*5
        self.acao_faq =min([max([0, self.acao_faq]),50])
        self.actions_faq.append(self.acao_faq)
        self.faq_RL = self.acao_faq

        self.acao_Fi += action[1]*5
        self.acao_Fi =min([max([0, self.acao_Fi]),50])
        self.actions_Fi.append(self.acao_Fi)
        self.Fi = self.acao_Fi

        self.acao_Fe += action[2]*5
        self.acao_Fe =min([max([0, self.acao_Fe]),50])
        self.actions_Fe.append(self.acao_Fe)
        self.Fe = self.acao_Fe

        self.integrator.step()
        self.t.append(self.integrator.t)
        self.current_step += 1
        y = self.integrator.y
        self.erro_vec_T.append((y[-2] - self.set_point_T)*self.K_t)
        self.erro_vec_etoh.append((y[2] - self.set_point_etoh)*self.K_oh)
        self.erro_vec_h.append((y[0] - self.set_point_h)*self.K_h)

        reward_T = self.compute_reward(self.erro_vec_T[-1])
        reward_etoh = self.compute_reward(self.erro_vec_etoh[-1])
        reward_h = self.compute_reward(self.erro_vec_h[-1])

        reward =  ((reward_T+ reward_etoh+ reward_h)/1.412) #[0,1]
        A = 1.2446
        B = -1.05
        a = 2.72197
        b = -0.22
        reward = A*(1/(1+np.exp(-(a*reward - b)))) + B


        #reward = reward - sum([abs(k) for k in action])/6

        reward = reward
        self.rw_vec_T.append(reward_T)
        self.rw_vec_etoh.append(reward_etoh)
        self.rw_vec_h.append(reward_h)
        self.rw_vec.append(reward)
        self.yeast.append(self.integrator.y[1])

        self.glic.append(self.integrator.y[3])
        self.oxy.append(self.integrator.y[4])
        self.T_jacket.append(self.integrator.y[6])
        self.cum_reward+=reward
        if (self.integrator.status != 'running' or
            y[-2]>50 or y[-2]<10 or
            y[0]>0.2865 or y[0]<0.0637
           ):
            done = True
            #print('done',self.current_step, y[-2], y[2], y[0])

        else:
            done = False

        obs =  [self.erro_vec_T[-1], self.erro_vec_h[-1], self.erro_vec_etoh[-1]]

        info = {'is_success': done}
        return obs, reward, done, info

    def render(self, mode='human'):
        print("Not implemented")
        return

    def close(self):
        print("Not implemented")
        return



In [ ]:
import random
import time

# random integer from 0 to 9

def evaluate_agent(param_vec, n_eval):
    param_  = [param_vec]*n_eval
    eval50 = []
    eval5 = []
    times = []
    k = random.randint(1000, 1000000)*10

    for [lr, gama, sig, steps_train,bat_size, t_freq, net_arch] in param_ :
            k=k+1
            start_time = time.time()
            env = bioreactor()
            #testar rw = min rw
            log_dir= str(k)+'ssv5_biorreactor_gama={1}_sig={2}_lr={0}_batch_size={3}_steps={4}_t_freq={5}_net_arch={6}/'.format(lr, gama, sig, bat_size, steps_train, t_freq,net_arch)
            print('...........................................................................'+log_dir)

            env = Monitor(env, log_dir)

            eval_env = reset_eval()
            eval_callback = EvalCallback(eval_env, best_model_save_path=log_dir,
                                         log_path=log_dir, eval_freq=int((steps_train/15)),
                                         deterministic=True, render=False,
                                        n_eval_episodes=5, verbose=0)

            n_actions = env.action_space.shape[-1]
            action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=sig * np.ones(n_actions))

            policy_kwargs = dict(net_arch=net_arch)

            model = DDPG("MlpPolicy",
                         env,
                         action_noise=action_noise,
                         verbose=0,
                        learning_rate=linear_schedule(lr),
                         buffer_size=1000000,
                         learning_starts=int(bat_size)*2,
                         batch_size=int(bat_size),
                         tau=0.005,
                         gamma=gama,
                         train_freq=(int(t_freq*125), 'step'),
                         gradient_steps=-1,
                         device='cuda',
                        policy_kwargs=policy_kwargs)

            model.learn(steps_train,callback=eval_callback)


            env = bioreactor()
            model = DDPG.load( log_dir+"best_model.zip")

            mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=50, deterministic =True)
            eval50.append(mean_reward)
            mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5, deterministic =True)
            eval5.append(mean_reward)
            times.append((time.time() - start_time))
            print(eval50[-1], eval5[-1], times[-1])
    print(eval50, eval5, times)
    return [eval50, eval5, times]

In [ ]:
res1 = evaluate_agent([0.00316,0.7,0.5,250000, 8000, 80, [64,56,32,12]], 10)


...........................................................................8739891ssv5_biorreactor_gama=0.7_sig=0.5_lr=0.00316_batch_size=8000_steps=250000_t_freq=80_net_arch=[56, 40, 16, 8]/
-0.18446119509229902 14.356329441070557 2895.3111670017242
...........................................................................8739892ssv5_biorreactor_gama=0.7_sig=0.5_lr=0.00316_batch_size=8000_steps=250000_t_freq=80_net_arch=[56, 40, 16, 8]/
